In [8]:
"imports"

import csv 
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold,train_test_split, LeaveOneOut,cross_val_score, cross_val_predict
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt 


"load data set"

dataset = pd.read_csv("C:/Users/nibha/Desktop/syria.csv") 
X = dataset.drop('status', axis=1) # seperate features 
y = dataset['status'] #seperate label

"convert categorical classes"

z=dataset['deathdate'] 
need_dummies=X.drop('deathdate', axis=1) 
data_with_dummies= pd.get_dummies(need_dummies) 
X_features = pd.concat([ data_with_dummies,z], axis=1) #combine categorical and non categorial features 
# X_features=X_features
# print(X_features.head(5))

X_features=np.asarray(X_features)

y = y.astype('category') 
# print(y.value_counts()) 
y_target = y.cat.codes 
# print(y_target.head(5)) 
# print(y.value_counts())

y_target=np.asarray(y_target)

"Leave one out"

l=len(X_features)

loo = LeaveOneOut()
loo.get_n_splits(X_features)

tree = DecisionTreeClassifier(max_depth = 5,random_state=1) 


i=1
for train_index, test_index in loo.split(X_features):
    
    tree.fit(X_features[train_index], y_target[train_index])
    lootarget_pred_real = tree.predict(X_features[test_index])
    
    i=i+1

lootarget_pred_prob = tree.predict_proba(X_features)[:,0]


def con(t_value,samples,target_pred_prob,y):
    target_pred=[]
    for j in range(samples):
      
        threshold=target_pred_prob[j]
        if (threshold<= t_value):
            target_pred.insert(j,1)
        else:
            target_pred.insert(j,0)
        
    tn, fp, fn, tp = confusion_matrix(y,target_pred).ravel() 
    
    sens= tp/(tp+fn) 
    spec= tn/(fp+tn) 
    acc=(tp+tn)/(tp+tn+fp+fn) 
    F1= (2*sens*spec)/(sens+spec)   
    
    return([tn,fp,fn,tp,sens,spec,acc,F1,target_pred])

samples=len(X_features)
loo_thresh0=con(0,samples,lootarget_pred_prob,y_target)
loo_thresh1=con(0.25,samples,lootarget_pred_prob,y_target)
loo_thresh2=con(0.5,samples,lootarget_pred_prob,y_target)
loo_thresh3=con(0.75,samples,lootarget_pred_prob,y_target)
loo_thresh4=con(1,samples,lootarget_pred_prob,y_target)

print(loo_thresh0[4])

0.00243902439024
